In [1]:
import os
import csv
import sys

csv.field_size_limit(sys.maxsize)

131072

###1. Load data: read in csv as a list of dictionaries

In [2]:
protests = []
with open('protests.csv', 'rU') as csvfile: # open file
    reader = csv.DictReader(csvfile) # create a reader
    for row in reader: # loop through rows
        protests.append(row) # append each row to the list

In [3]:
#protests[:2]

In [4]:
type(protests)

list

In [5]:
keys = protests[1].keys()
keys

['', 'PUBLICATION', 'TITLE', 'COUNTRY', 'LENGTH', 'TEXT', 'DATE']

###2. Extract text of articles to prepare to loop through

In [6]:
articles = [li['TEXT'] for li in protests]

In [7]:
#articles[:2]
len(articles)

1429

###3. Write function that passes two arguments: article and city
Function will search the text of the article for the city of interest, identify its index, and then search for the root words "protest" or its synonym "manifest" (short for manifestação, and permutations thereof) within 30 words of the city name.

In [45]:
def query_cities(article, city):

    text = article.split()
    if city in text:
        takeIndex = text.index(city)
        wc = len(text)
        if takeIndex > 30 and takeIndex < wc - 30:
            subset = ' '.join(text[takeIndex-30:takeIndex+30])
        elif takeIndex < 30 and takeIndex < wc - 30:
            subset = ' '.join(text[:takeIndex+30])
        elif takeIndex > 30 and takeIndex > wc - 30:
            subset = ' '.join(text[takeIndex-30:])

        if 'protest'or 'manifest' in subset:
            return(1)
    else:
        return(0)

###4. Play with some test cases to see what the function's blind spots might be

In [46]:
test_article = articles[339]
print test_article

Para o antropólogo Paulo Storani, ex-capitão do Batalhão de Operações Especiais (Bope) da PM, as autoridades de segurança estão perdendo a "guerra da comunicação" na defesa do programa das UPPs do Rio. Recentemente está se tornando comum protestos de moradores de favelas pacificadas exigindo a retirada das UPPs destes locais. Isso ocorreu nas várias manifestações de moradores do Pavão-Pavãozinho, em Copacabana (zona sul), contra a morte do dançarino do programa Esquenta, da TV Globo Douglas Rafael Pereira. "Nestes protestos estão surgindo várias faixas com os dizeres 'fora UPP'. Será que isso é uma exigência da maioria das pessoas que vivem ali, ou apenas de um grupo de moradores que ficaram sem renda com a saída dos traficantes? É sabido que muitas pessoas que não necessariamente estavam ligadas ao tráfico eram contratadas para fazer a endolação, ou seja, preparar a droga para venda", explica Storani. "Está faltando coragem de as autoridades assumirem os erros do projeto, talvez por s

In [47]:
test_article2 = articles[1229]
print test_article2

Os deputados estaduais de São Paulo aprovaram em votação simbólica, na noite de anteontem, um projeto que proíbe "o uso de máscara ou qualquer outro paramento que possa ocultar o rosto da pessoa, ou que dificulta ou impeça a sua identificação" nos protestos de rua. De autoria do deputado Campos Machado (PTB), o projeto de lei só exclui da proibição as "manifestações e as reuniões culturais no calendário oficial do Estado". O objetivo é inibir a atuação de manifestantes mascarados ligados à tática black bloc. Ele segue para o governador Geraldo Alckmin (PDSB), que tem o prazo de 15 dias para sancioná-lo ou vetá-lo. Ainda conforme o texto, a plataforma principal de reivindicação desse grupo é "destruir, danificar, explodir, queimar, saquear e aterrorizar". "Tal comportamento tem esvaziado as legítimas manifestações e prejudicado o direito dos demais cidadãos de bem de se manifestar. Além, por óbvio, de deixar rastros de pânico e destruição e, consequentemente, causando prejuízos ao erári

In [48]:
test_article3 = articles[201]
print test_article3

O clima de disputa política entre a presidente Dilma Rousseff (PT) e o ex-governador de Pernambuco e presidenciável Eduardo Campos (PSB) é expresso logo na entrada de Serra Talhada, município sertanejo a 415 quilômetros do Recife, onde a presidente inaugura nesta tarde de segunda-feira, 14, primeira etapa da Adutora do Pajeú - obra que já havia sido parcialmente inaugurada por ela e pelo agora opositor Campos, em março do ano passado. Quase lado a lado, dois outdoors expressam feitos dos dois governos: "Dilma é apoio aos agricultores, obrigado pelos equipamentos", diz um, enquanto o outro fala de "mais uma ação do governo do Estado na saúde", com a chegada de medicamentos excepcionais. "O clima está acirrado", reconhece o prefeito do PT, Luciano Duque, que disse ter pago, do próprio bolso, três outdoors para recepcionar a presidente. Cada um custou R$ 800,00. O outro aborda o programa Minha Casa, Minha Vida, com 899 casas já construídas, e o terceiro, a atenção ao ensino infantil e fun

In [59]:
#the function fails to find the word "protest" within 30 words of the city "Rio" in test_article
query_cities(test_article, "Rio") 

0

In [60]:
#referring to the text, we see that this is because Rio is at the end of a sentance and has a '.' after it \
#when we add said period, the function works
query_cities(test_article,"Rio.")

1

In [61]:
#the function fails to find the word "protest" within 30 words of the compound city "São Paulo" in test_article2
query_cities(test_article2, "São Paulo")

0

In [62]:
#but correctly identifies it for the single word "Paulo"
query_cities(test_article2, "Paulo")

1

In [63]:
# and for the single word "São", which shows that the function is not accent sensitive, either
query_cities(test_article2, "São")

1

In [64]:
# test case 3 also confirms that the accents are fine (Brasília is recognized just the same) \
# but reveals the same punctuation problem as case one, which missed "Rio." \
# This one misses "Brasília,"
query_cities(test_article3, "Brasília")

0

In [65]:
query_cities(test_article3, "Brasília,")

1

###Review special cases that I'll need to account for before running the for loop through all the articles:

| Condition       | Example          | Works?  |
|------------- |-------------| -----|
| accent      | "Brasília" | yes |
| punctuation      | "Rio."      | no    |
| compound | "São Paulo"      |    no |

###5. Call function on each of the 10 cities
***Need to figure out how to store these to later cbind to the original protests.csv...***

In [71]:
for item in articles:
    query_cities(item, "Rio")

In [72]:
for item in articles:
    query_cities(item, "Curitiba")

In [73]:
for item in articles:
    query_cities(item, "Belo Horizonte")

In [74]:
for item in articles:
    query_cities(item, "São Paulo")

In [75]:
for item in articles:
    query_cities(item, "Campinas")

In [76]:
for item in articles:
    query_cities(item, "Salvador")

In [77]:
for item in articles:
    query_cities(item, "Recife")

In [78]:
for item in articles:
    query_cities(item, "Fortaleza")

In [79]:
for item in articles:
    query_cities(item, "Brasília")

In [80]:
for item in articles:
    query_cities(item, "Porto Alegre")

###6. Make city dictionary
if query_cities returns True ("1")
add a new Key:

In [154]:
for d in articles:
    d['Rio'] = query_cities(d['TEXT'],"Rio")
    d['Curitiba'] = query_cities(d['TEXT'],"Curitiba")
    d['Belo Horizonte'] = query_cities(d['TEXT'],"Belo Horizonte")
    d['São Paulo'] = query_cities(d['TEXT'],"São Paulo")
    d['Salvador'] = query_cities(d['TEXT'],"Salvador")
    d['Porto Alegre'] = query_cities(d['TEXT'],"Porto Alegre")
    d['Fortaleza'] = query_cities(d['TEXT'],"Fortaleza")   
    d['Recife'] = query_cities(d['TEXT'],"Recife")
    d['Campinas'] = query_cities(d['TEXT'],"Campinas")
    d['Brasília'] = query_cities(d['TEXT'],"Brasília")

TypeError: string indices must be integers, not str

###7. Write CSV

In [67]:
#writing column headings
keys = cities[0].keys()
keys

NameError: name 'cities' is not defined

In [68]:
#writing the rest
with open('../Data/upr-info.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(cities)

NameError: name 'cities' is not defined